Ebook: A Hands-On Guide to Biomechanics Data Analysis with Python and AI
Author: Dr. Hossein Mokhtarzadeh
Powered by PoseIQ™

This notebook loads sample biomechanics data and shows how to bring it into Python.
Click Runtime → Run all.


In [ ]:
!pip install ezc3d pandas

In [ ]:
!wget https://raw.githubusercontent.com/hmok/BiomechPythonAI_Guide/main/notebooks/Chapter1Input.py -O Chapter1Input.py
new_var = %run /content/Chapter1Input.py

In [ ]:
# ============================
# Forces & moments per plate (no zero-clipping; robust vertical detection)
# ============================
import numpy as np
import matplotlib.pyplot as plt

P = c3d["parameters"]

# Rates / sizes
ptHz    = float(P["POINT"]["RATE"]["value"][0])            # 50
anHz    = float(P["ANALOG"]["RATE"]["value"][0])           # 200
nFrames = int(c3d["data"]["points"].shape[2])              # 450
subf    = int(round(anHz / ptHz))                          # 4
t       = np.arange(nFrames, dtype=float) / ptHz

# Analog cube: (subframes, nAnalogChannels, nPointFrames)
A = c3d["data"]["analogs"]

# CHANNEL map: 6 x nPlates (1-based indices into ANALOG)
chan = np.array(P["FORCE_PLATFORM"]["CHANNEL"]["value"]).reshape(6, -1)
nPlates = chan.shape[1]
chan0 = np.clip(chan - 1, 0, A.shape[1]-1)                 # to 0-based, safety-clipped

# Optional CAL_MATRIX (per plate 6x6)
CAL = None
if "CAL_MATRIX" in P["FORCE_PLATFORM"]:
    raw = np.array(P["FORCE_PLATFORM"]["CAL_MATRIX"]["value"])
    if raw.size == 36 * nPlates:
        CAL = raw.reshape(nPlates, 6, 6)
    elif raw.size == 36:
        CAL = np.tile(raw.reshape(1,6,6), (nPlates,1,1))

def collapse_subframes(arr_subf_6_frames):
    """
    Average analog subframes -> point frames.
    Accepts typical shapes: (subf, 6, nFrames), (1, 6, nFrames*subf), etc.
    Returns (6, nFrames).
    """
    arr = np.asarray(arr_subf_6_frames)
    # Common case
    if arr.ndim == 3 and arr.shape[0] == subf and arr.shape[2] == nFrames:
        return arr.mean(axis=0)
    # (1,6,nFrames*subf)
    if arr.ndim == 3 and arr.shape[0] == 1 and arr.shape[2] == nFrames*subf:
        return arr.reshape(1,6,nFrames,subf).mean(axis=3)[0]
    # (subf, nFrames) per channel - reduce
    if arr.ndim == 3 and arr.shape[1] == 6:
        # collapse subframes by mean along axis 0
        return arr.mean(axis=0)
    # Fallback: interpolate each of the 6 channels to point time
    out = []
    for k in range(6):
        y = arr[:, k, :].ravel()
        t_an = np.linspace(0, (len(y)-1)/anHz, len(y))
        out.append(np.interp(t, t_an, y))
    return np.vstack(out)

def pick_vertical_and_flip(F_3xN):
    """
    Pick vertical axis as the one with the largest robust magnitude (95th percentile of |F|).
    Flip sign if the series is mostly negative (so positive is 'up').
    Return (Fx, Fy, Fz_corrected, vert_axis_index).
    """
    mags = [np.percentile(np.abs(F_3xN[i]), 95) for i in range(3)]
    v = int(np.argmax(mags))           # 0=X, 1=Y, 2=Z
    Fv = F_3xN[v].astype(float)
    # Flip if mostly negative (median < 0)
    if np.nanmedian(Fv) < 0:
        Fv = -Fv
        # also flip the chosen axis in the 3xN for consistency
        F = F_3xN.copy()
        F[v] = Fv
    else:
        F = F_3xN
    return F[0], F[1], F[2], v

per_plate = []
for p in range(nPlates):
    # pull the six mapped channels for this plate
    raw6 = A[:, chan0[:, p], :]                         # (subf, 6, nFrames) in lab units
    six_by_frames = collapse_subframes(raw6)            # (6, nFrames)
    # apply CAL if available
    if CAL is not None:
        six_by_frames = (CAL[p] @ six_by_frames).astype(float)

    forces  = six_by_frames[0:3, :]                     # Fx,Fy,Fz (lab coords)
    moments = six_by_frames[3:6, :]                     # Mx,My,Mz

    Fx, Fy, Fz_corr, vert_axis = pick_vertical_and_flip(forces)
    per_plate.append(dict(Fx=Fx, Fy=Fy, Fz=Fz_corr,
                          Mx=moments[0], My=moments[1], Mz=moments[2],
                          vert_axis=vert_axis))

# --------- Plot (compare directly to Mokka) ---------
axis_name = ["X","Y","Z"]
for i, d in enumerate(per_plate, start=1):
    # Forces (no zero-clipping; just flipped if needed)
    plt.figure(figsize=(10,4))
    plt.plot(t, d["Fx"], label="Fx")
    plt.plot(t, d["Fy"], label="Fy")
    plt.plot(t, d["Fz"], label="Fz (vertical)")
    plt.title(f"Plate {i} -- Forces (vertical axis picked: {axis_name[d['vert_axis']]})")
    plt.xlabel("Time (s)"); plt.ylabel("Force (lab units)"); plt.legend(); plt.tight_layout(); plt.show()

    # Moments (as-is)
    plt.figure(figsize=(10,4))
    plt.plot(t, d["Mx"], label="Mx")
    plt.plot(t, d["My"], label="My")
    plt.plot(t, d["Mz"], label="Mz")
    plt.title(f"Plate {i} -- Moments")
    plt.xlabel("Time (s)"); plt.ylabel("Moment (lab units)"); plt.legend(); plt.tight_layout(); plt.show()

    def _pk(v): return float(np.nanmax(np.abs(v))) if v is not None and len(v) else np.nan
    print(f"Plate {i} peaks (abs) -> Fx:{_pk(d['Fx']):.1f}  Fy:{_pk(d['Fy']):.1f}  "
          f"Fz:{_pk(d['Fz']):.1f}   Mx:{_pk(d['Mx']):.1f}  My:{_pk(d['My']):.1f}  Mz:{_pk(d['Mz']):.1f}")

Step 1: Detect gait events (HS/TO) from vertical GRF
We use a hysteresis rule (two thresholds + min stance). Because units may differ (Newtons vs “nt”), we default to percent-of-peak thresholds so it’s robust either way. You can also force Newton thresholds if you know forces are calibrated.


In [ ]:
# ===========================================
# Step 1 -- Detect HS/TO from vertical GRF
# (robust to vertical axis = X/Y/Z; no scaling)
# ===========================================
import numpy as np
import matplotlib.pyplot as plt

# Timing
ptHz    = float(c3d["parameters"]["POINT"]["RATE"]["value"][0])
nFrames = int(c3d["data"]["points"].shape[2])
t       = np.arange(nFrames, dtype=float) / ptHz

# Build a stack of vertical forces from Step A (use vert_axis to pick the right component)
def _vertical_series(d):
    va = int(d["vert_axis"])
    if va == 0: return np.asarray(d["Fx"], float)  # X is vertical
    if va == 1: return np.asarray(d["Fy"], float)  # Y is vertical
    return np.asarray(d["Fz"], float)              # Z is vertical

Fv_stack = np.vstack([_vertical_series(d) for d in per_plate])  # (nPlates, nFrames)

# Choose plate with earliest HS AFTER a small start buffer (avoids HS at frame 0)
start_buffer_s = 1.0
buf_fr = int(round(start_buffer_s * ptHz))

def hysteresis_events(F, fs, on, off, min_contact_s=0.10):
    """Two-threshold ON/OFF detector with minimum stance-duration constraint."""
    state = False
    contact = np.zeros_like(F, dtype=bool)
    for i, v in enumerate(F):
        if not state and v > on: state = True
        elif state and v < off:  state = False
        contact[i] = state
    edges = np.diff(contact.astype(np.int8), prepend=0)
    hs = np.where(edges == 1)[0]
    to = np.where(edges == -1)[0]

    HS, TO, j = [], [], 0
    min_samp = int(round(min_contact_s * fs))
    for h in hs:
        while j < len(to) and to[j] <= h: j += 1
        if j < len(to) and (to[j] - h) >= min_samp:
            HS.append(int(h)); TO.append(int(to[j])); j += 1
    return np.array(HS, int), np.array(TO, int)

# Auto thresholds: % of robust peak (unit-agnostic; good for beginners)
def auto_thresholds(F):
    pk = np.percentile(F, 95)
    return 0.10 * pk, 0.06 * pk    # ON=10% of robust peak, OFF=6%

# Find earliest valid HS after buffer for each plate; pick that plate
candidates = []
events_by_plate = []
for p in range(Fv_stack.shape[0]):
    on, off = auto_thresholds(Fv_stack[p])
    HS, TO  = hysteresis_events(Fv_stack[p], ptHz, on, off, min_contact_s=0.10)
    HS_buf  = HS[HS >= buf_fr]
    first_buf = int(HS_buf[0]) if HS_buf.size else None
    candidates.append((p, first_buf, on, off))
    events_by_plate.append((HS, TO))

eligible = [(p, fb) for (p, fb, on, off) in candidates if fb is not None]
if eligible:
    plate_idx = min(eligible, key=lambda x: x[1])[0]
else:
    fallback = [(p, ev[0][0]) for p, ev in enumerate(events_by_plate) if len(ev[0])]
    plate_idx = min(fallback, key=lambda x: x[1])[0] if fallback else 0

HS, TO = events_by_plate[plate_idx]
hs_t, to_t = HS / ptHz, TO / ptHz

print(f"Selected plate: {plate_idx+1}/{Fv_stack.shape[0]}")
print("HS frames:", HS.tolist(), "| HS times (s):", np.round(hs_t, 3).tolist())
print("TO frames:", TO.tolist(), "| TO times (s):", np.round(to_t, 3).tolist())

# Overlay plot
plt.figure(figsize=(10,4))
plt.plot(t, Fv_stack[plate_idx], linewidth=1.5, label=f"Plate {plate_idx+1} vertical GRF")
plt.plot(t[HS], Fv_stack[plate_idx][HS], "^", markersize=8, label="HS")
plt.plot(t[TO], Fv_stack[plate_idx][TO], "v", markersize=8, label="TO")
plt.title("HS/TO from vertical GRF (hysteresis)")
plt.xlabel("Time (s)"); plt.ylabel("Force (lab units)")
plt.legend(); plt.tight_layout(); plt.show()


Why this works for beginners
It doesn’t assume Z-up; uses the axis you detected in Step A (vert_axis) so the vertical curve matches Mokka.
Percent-of-peak thresholds make it robust whether your forces are Newtons or uncalibrated analog units.
The start buffer prevents reporting HS at frame 0 when the foot starts on the plate.


If you already know which plate is the right leg, you can replace the selection logic with plate_idx = 0 (or 1, etc.) right before the print/plot block.
What readers should look for
Frame count (e.g., 450) and duration (e.g., 450/50 = 9.0 s) match the metadata.
The selected plate shows HS roughly where expected (e.g., ~frame 270 at 50 Hz ⇒ ~5.4 s).
If your trial starts with the foot already on a plate, the start buffer avoids HS at frame 0.


Step 1: Detect Gait Events (Heel Strike & Toe-Off)
We detect foot–ground contact from the vertical ground reaction force (GRF). A heel strike (HS) is when GRF rises above an ON threshold after swing; a toe-off (TO) is when it falls below an OFF threshold. We use a simple hysteresis rule (two thresholds + minimum stance duration) for robustness, and fall back to heel marker kinematics if force data can’t give clean events. The cell below prints HS/TO in frames and seconds and tags them in the data for later plots.
love it—here’s a clean, beginner-friendly write-up + code for your book’s Chapter 3 Steps 2–4, matched to the variables and flow you already have from Step A (forces per plate) and Step 1 (HS/TO).
 It assumes you’ve just run Step A (which created per_plate) and Step 1 (which created Fv_stack, plate_idx, HS, TO, ptHz, nFrames).


Step 2 — Calculate key metrics
What we’ll compute
Peak vertical GRF (in the same units you just plotted; if your data are in Newtons and you know body mass, we’ll also show BW).
Stride time and cadence from HS events.
Stance summaries (HS→TO segments with duration and peak force).
Symmetry (peak-to-peak) across the two plates (only if both plates look realistic).

In [ ]:
# ============================
# Step 2 -- Key metrics
# ============================
import numpy as np
import pandas as pd

# Pull the selected plate's vertical GRF (the same one used in Step 1)
Fv = Fv_stack[plate_idx].astype(float)              # (nFrames,)
t  = np.arange(nFrames, dtype=float) / ptHz

# --- Peak vertical GRF ---
peak_force = float(np.nanmax(Fv))                   # units = whatever your forces are in
print(f"Peak vertical GRF (selected plate): {peak_force:.1f} (lab units)")

# Optional: Body-weight normalization if you know mass (and forces are Newtons)
if 'body_mass_kg' in globals() and isinstance(body_mass_kg, (int, float)) and body_mass_kg > 0:
    peak_bw = peak_force / (body_mass_kg * 9.81)
    print(f"...which is {peak_bw:.2f} BW")

# --- Stride time & cadence from HS events ---
hs_t = HS / ptHz
if len(hs_t) > 1:
    stride_times = np.diff(hs_t)                    # seconds
    mean_stride  = float(np.mean(stride_times))
    cadence_spm  = 120.0 / mean_stride             # steps/min (2 steps per stride)
    print(f"Average stride time: {mean_stride:.3f} s (n={len(stride_times)})")
    print(f"Approx cadence: {cadence_spm:.1f} steps/min")
else:
    print("Not enough HS events to compute stride time/cadence.")

# --- Build stance summary (HS→TO for selected plate) ---
stance_rows = []
j = 0
for h in HS:
    while j < len(TO) and TO[j] <= h:
        j += 1
    if j < len(TO) and TO[j] > h:
        h_frame = int(h)
        to_frame = int(TO[j])
        seg = slice(h_frame, to_frame)
        stance_rows.append({
            "HS_frame": h_frame,
            "TO_frame": to_frame,
            "HS_time_s": float(h_frame/ptHz),
            "TO_time_s": float(to_frame/ptHz),
            "stance_dur_s": float((to_frame - h_frame)/ptHz),
            "peak_vertical_in_stance": float(np.nanmax(Fv[seg])),
        })
        j += 1

stance_summary = pd.DataFrame(stance_rows)
print("\nStance summary (first rows):")
try:
    from IPython.display import display
    display(stance_summary.head())
except:
    print(stance_summary.head())

# --- Symmetry (peak-to-peak) across plates (only if both plates look reasonable) ---
def _robust_peak(v):  # robust to spikes
    return float(np.percentile(np.abs(v), 99))

symmetry_index = None
if Fv_stack.shape[0] >= 2:
    pk0 = _robust_peak(Fv_stack[0])
    pk1 = _robust_peak(Fv_stack[1])
    denom = 0.5 * (pk0 + pk1)
    if denom > 1e-8:
        symmetry_index = 100.0 * abs(pk0 - pk1) / denom
        print(f"\nGRF symmetry (peak-to-peak, robust): {symmetry_index:.1f}%")
    else:
        print("\nSymmetry not computed (invalid denominator).")
else:
    print("\nSymmetry not computed (need ≥2 plates).")


Step 3: Segment steps & cycles Across-plate steps & per-plate stance
What we’ll do
Use consecutive HS→HS windows to define gait cycles.
(You already have stance windows from HS→TO in Step 2.)

In [ ]:
# ===========================================
# Step 3 -- Across-plate step cycles + per-plate stance
# ===========================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Inputs from Steps A & 1:
# - Fv_stack: (nPlates, nFrames) vertical force per plate (already correct axis/sign)
# - ptHz, nFrames
t = np.arange(nFrames, dtype=float) / ptHz
nPlates = Fv_stack.shape[0]

# --- same hysteresis as Step 1 ---
def hysteresis_events(F, fs, on, off, min_contact_s=0.10):
    state=False
    contact = np.zeros_like(F, dtype=bool)
    for i,v in enumerate(F):
        if not state and v > on: state=True
        elif state and v < off:  state=False
        contact[i] = state
    edges = np.diff(contact.astype(np.int8), prepend=0)
    hs = np.where(edges==1)[0]; to = np.where(edges==-1)[0]
    HS, TO, j = [], [], 0
    min_samp = int(round(min_contact_s*fs))
    for h in hs:
        while j < len(to) and to[j] <= h: j += 1
        if j < len(to) and (to[j] - h) >= min_samp:
            HS.append(int(h)); TO.append(int(to[j])); j += 1
    return np.array(HS, int), np.array(TO, int)

def auto_thresholds(F):
    pk = np.percentile(F, 95)      # robust peak
    return 0.10*pk, 0.06*pk        # ON/OFF as % of peak (unit-agnostic)

# 1) Detect events on EACH plate
events = []   # list of dicts per plate
for p in range(nPlates):
    on, off = auto_thresholds(Fv_stack[p])
    HS, TO  = hysteresis_events(Fv_stack[p], ptHz, on, off, min_contact_s=0.10)
    events.append(dict(HS=HS, TO=TO, on=on, off=off))

# 2) Per-plate stance table (HS→TO within the same plate)
stance_rows = []
for p, ev in enumerate(events, start=1):  # 1-based plate index for readability
    HS, TO = ev["HS"], ev["TO"]
    j = 0
    for h in HS:
        while j < len(TO) and TO[j] <= h:
            j += 1
        if j < len(TO) and TO[j] > h:
            to = TO[j]
            seg = slice(h, to)
            stance_rows.append({
                "plate": p,
                "HS_frame": int(h), "TO_frame": int(to),
                "HS_time_s": float(h/ptHz), "TO_time_s": float(to/ptHz),
                "stance_dur_s": float((to-h)/ptHz),
                "peak_vertical_in_stance": float(np.nanmax(Fv_stack[p-1][seg])),
            })
            j += 1

stance_summary = pd.DataFrame(stance_rows).sort_values(["HS_frame"]).reset_index(drop=True)

# 3) Across-plate step cycles (HS_k -> HS_{k+1}, regardless of plate)
#    This captures plate 2 → plate 1 as a valid step interval.
hs_all = []
for p, ev in enumerate(events, start=1):
    for h in ev["HS"]:
        hs_all.append((int(h), float(h/ptHz), p))
hs_all.sort(key=lambda x: x[0])   # sort by frame

step_rows = []
for k in range(len(hs_all)-1):
    h1_fr, h1_t, p1 = hs_all[k]
    h2_fr, h2_t, p2 = hs_all[k+1]
    step_rows.append({
        "from_plate": p1,
        "to_plate": p2,
        "HS1_frame": h1_fr, "HS2_frame": h2_fr,
        "HS1_time_s": h1_t,  "HS2_time_s": h2_t,
        "step_dur_s": float(h2_t - h1_t),
    })

step_summary = pd.DataFrame(step_rows)

# --- Reports ---
print("Per-plate stance (first rows):")
try:
    from IPython.display import display
    display(stance_summary.head())
except:
    print(stance_summary.head())

print("\nAcross-plate step cycles (HS→HS) (first rows):")
try:
    display(step_summary.head())
except:
    print(step_summary.head())

if len(step_summary):
    mean_step = float(step_summary["step_dur_s"].mean())
    cadence_spm = 60.0 / mean_step if mean_step > 0 else np.nan
    print(f"\nExtracted {len(step_summary)} HS→HS steps across plates.")
    print(f"Mean step time: {mean_step:.3f} s  |  Cadence (approx): {cadence_spm:.1f} steps/min")
else:
    print("\nNo across-plate HS→HS steps found -- check thresholds or plate mapping.")

# --- Quick visualization: both plates with HS markers ---
plt.figure(figsize=(11,4))
colors = ["C0","C1","C2","C3"]
for p in range(nPlates):
    plt.plot(t, Fv_stack[p], label=f"Plate {p+1} vertical")
    if len(events[p]["HS"]):
        plt.plot(t[events[p]["HS"]], Fv_stack[p][events[p]["HS"]],
                 "^", markersize=7, color=colors[p%len(colors)], label=f"P{p+1} HS")
plt.xlabel("Time (s)"); plt.ylabel("Vertical force (lab units)")
plt.title("Vertical GRF per plate with HS markers")
plt.legend(); plt.tight_layout(); plt.show()

Step 4: Apply filters (Butterworth & Savitzky–Golay)
Why filter?
To reduce high-frequency noise before differentiating or event logic from kinematics. We’ll show:
A 4th-order Butterworth low-pass (zero-phase) for the heel marker Z and for the vertical GRF.
A Savitzky–Golay smoothed heel Z as an alternative (works on shorter segments too).

In [ ]:
# ============================
# Step 4 -- Smooth signals
# ============================
import numpy as np
from scipy.signal import butter, filtfilt, savgol_filter
import matplotlib.pyplot as plt

# --- Helper: safe low-pass ---
def lowpass_zero_phase(x, fs, fc, order=4):
    x = np.asarray(x, float)
    nyq = fs/2.0
    fc  = min(fc, nyq*0.9)
    b, a = butter(order, fc/nyq, btype="low")
    padlen = 3 * (max(len(a), len(b)) - 1)  # e.g., 15 for order 4
    if x.ndim == 1 and x.size <= padlen:
        # short vector fallback: just return SavGol
        win = min(11, x.size - 1 if x.size % 2 == 0 else x.size)
        win = max(5, win | 1)  # make odd
        return savgol_filter(x, window_length=win, polyorder=3)
    return filtfilt(b, a, x, axis=0)

# --- Smooth vertical GRF (selected plate) ---
force_fc_hz = 20.0
Fv_raw = Fv_stack[plate_idx].astype(float)
Fv_filt = lowpass_zero_phase(Fv_raw, fs=ptHz, fc=force_fc_hz)

plt.figure(figsize=(10,4))
plt.plot(t, Fv_raw,  label="vertical force (raw)",  linewidth=1)
plt.plot(t, Fv_filt, label=f"vertical force (Butterworth {force_fc_hz:.0f} Hz)", linewidth=2)
plt.xlabel("Time (s)"); plt.ylabel("Force (lab units)")
plt.title("Vertical GRF -- raw vs filtered")
plt.legend(); plt.tight_layout(); plt.show()

# --- Smooth heel marker Z (if available from earlier chapters) ---
marker_fc_hz = 6.0
if 'df_markers_mm' in globals():
    heel_z_m = (df_markers_mm["Heel_Z_mm"].to_numpy().astype(float) / 1000.0)  # mm → m
    heel_z_lp = lowpass_zero_phase(heel_z_m, fs=ptHz, fc=marker_fc_hz)
    # Savitzky-Golay alternative
    w = min(21, len(heel_z_m) - 1 if len(heel_z_m) % 2 == 0 else len(heel_z_m))
    w = max(5, w | 1)  # odd
    heel_z_sg = savgol_filter(heel_z_m, window_length=w, polyorder=3)

    plt.figure(figsize=(10,4))
    plt.plot(t, heel_z_m,  label="Heel Z (raw m)", linewidth=1)
    plt.plot(t, heel_z_lp, label=f"Heel Z (Butterworth {marker_fc_hz:.0f} Hz)", linewidth=2)
    plt.plot(t, heel_z_sg, label="Heel Z (Savitzky-Golay)", linewidth=2)
    plt.xlabel("Time (s)"); plt.ylabel("Position (m)")
    plt.title("Heel Z -- raw vs filtered")
    plt.legend(); plt.tight_layout(); plt.show()
else:
    print("Heel marker table (df_markers_mm) not found -- skipping heel kinematics smoothing.")

What beginners should check
Frame count & duration: e.g., 450 frames at 50 Hz → 9.0 s total.
Vertical force shape: stance lobes align with HS/TO markers; peaks look reasonable.
Metrics sanity: stride ~1–1.5 s (walking), cadence ~80–120 spm, stance durations ~0.5–0.8 s.
Symmetry: low % (e.g., <10–15%) for healthy symmetric walking; higher indicates asymmetry.
When you’re happy with Steps 2–4, we can add a tiny “export CSVs” snippet (timeseries, stance/cycle tables) and move to Chapter 4: visualize.
Summary
In this chapter you:
Detected gait events from vertical force
Computed peak GRF, stride time, and symmetry
Segmented data into individual steps
Smoothed signals for cleaner analysis
